# 02_baseline_rule_based — Binario A/D

**Objetivo:** baseline **rule-based** usando el fork del proyecto colombiano (solo **Ansiedad/Depresión**) para obtener una primera línea de referencia.  
**Justificación:** las reglas permiten:
- establecer un punto de partida interpretable (trazabilidad por JSON/patrones),
- detectar fallos sistemáticos del dataset (typos, negación, expresiones locales),
- guiar el diseño del *cleaning* y la selección de fenotipos relevantes para A/D.

> Nota: mantenemos **preprocesamiento ligero** para no romper *ConText* ni *TargetMatcher*.


In [19]:
# ===============================================================
# Setup: Paths, Imports, y Utilidades Compartidas
# ===============================================================

from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Intentar importar utilidades compartidas
try:
    import sys
    sys.path.insert(0, str(Path.cwd().parent))
    from utils_shared import setup_paths, guess_text_col, guess_label_col, normalize_label
    print("✅ Utilizando utils_shared.py")
    
    # Setup de paths centralizado
    paths = setup_paths()
    BASE_PATH = paths['BASE_PATH']
    DATA_PATH = paths['DATA_PATH']
    FORK_PATH = paths['FORK_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    
    # Usar funciones centralizadas
    _guess_text_col = guess_text_col
    _guess_label_col = guess_label_col
    _norm_label_bin = normalize_label
    
except ImportError:
    print("⚠️ utils_shared.py no encontrado, usando funciones locales")
    
    # Setup manual de paths
    BASE_PATH = Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    
    DATA_PATH = BASE_PATH / "data"
    FORK_PATH = BASE_PATH / "Spanish_Psych_Phenotyping_PY"
    SPLITS_PATH = DATA_PATH / "splits"
    
    DATA_PATH.mkdir(exist_ok=True)
    
    # Funciones helper locales
    def _guess_text_col(df):
        for c in ["texto", "text", "comment", "comentario"]:
            if c in df.columns:
                return c
        return df.columns[0]
    
    def _guess_label_col(df):
        for c in ["etiqueta", "label", "category"]:
            if c in df.columns:
                return c
        return df.columns[1] if len(df.columns) > 1 else df.columns[-1]
    
    def _norm_label_bin(s):
        if pd.isna(s): 
            return ""
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
        return {'depresivo': 'depresion'}.get(s, s)

# Validar existencia de directorios críticos
if not FORK_PATH.exists():
    raise FileNotFoundError(
        f"❌ Fork no encontrado en {FORK_PATH}\n"
        f"   Este baseline requiere Spanish_Psych_Phenotyping_PY/"
    )

if not SPLITS_PATH.exists():
    raise FileNotFoundError(
        f"❌ Splits no encontrados en {SPLITS_PATH}\n"
        f"   Debes ejecutar primero: 02_create_splits.ipynb"
    )

print(f"✅ Paths configurados:")
print(f"   BASE_PATH:   {BASE_PATH}")
print(f"   DATA_PATH:   {DATA_PATH}")
print(f"   FORK_PATH:   {FORK_PATH}")
print(f"   SPLITS_PATH: {SPLITS_PATH}")

# Columnas esperadas en dataset_base.csv
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

✅ Utilizando utils_shared.py
✅ Paths configurados:
   BASE_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay
   DATA_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
   FORK_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY
   SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y preprocesamiento **ligero** (conserva tildes y casing, colapsa alargamientos)

In [20]:
# ===============================================================
# Carga de Datos y Preprocesamiento LIGERO (Rule-Based)
# ===============================================================
#
# ESTRATEGIA DE PREPROCESAMIENTO: CONSERVADORA (mínimo)
#
# ¿Por qué preprocesamiento ligero?
#
# 1. **Preserva tildes y mayúsculas**:
#    - Los patrones JSON del fork usan "Depresión", "Ansiedad" con tildes
#    - ConText y TargetMatcher son case-sensitive en algunos contextos
#    - Normalizar agresivamente = romper el matching de patrones
#
# 2. **Solo colapsa alargamientos**:
#    - "holaaa" → "holaa" (mantiene énfasis sin perder matching)
#    - Usuarios escriben "estabaaa tristeee" con múltiples letras
#    - Patrones no capturan "tristeeeee" pero sí "tristee"
#
# 3. **Normaliza espacios**:
#    - Colapsa múltiples espacios en uno solo
#    - Elimina espacios al inicio/final
#    - No afecta matching pero limpia entrada
#
# ¿Qué NO hace este preprocesamiento?
# - ❌ No convierte a minúsculas (rompe patrones)
# - ❌ No elimina tildes/acentos (rompe patrones)
# - ❌ No elimina puntuación (ConText la usa)
# - ❌ No hace stemming/lemmatización (no necesario para reglas)
#
# Comparación con otros baselines:
# - TF-IDF: Preprocesamiento agresivo (lowercase, sin tildes, sin puntuación)
# - BETO: Solo tokenización (el modelo maneja mayúsculas/tildes)
# - Rule-Based: Conservador (preserva estructura original)
#
# ===============================================================

import pandas as pd, re, unicodedata

# Cargar splits unificados desde 02_create_splits.ipynb
dataset_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')
train_indices = pd.read_csv(SPLITS_PATH / 'train_indices.csv')['row_id'].values
val_indices = pd.read_csv(SPLITS_PATH / 'val_indices.csv')['row_id'].values

print(f"✅ Splits cargados desde {SPLITS_PATH}/:")
print(f"   Train: {len(train_indices)} ejemplos")
print(f"   Val:   {len(val_indices)} ejemplos")
print(f"   Total: {len(dataset_base)} ejemplos en dataset_base.csv")

# Detectar columnas automáticamente
text_col = _guess_text_col(dataset_base)
label_col = _guess_label_col(dataset_base)
print(f"\n📋 Columnas detectadas: texto='{text_col}', label='{label_col}'")

# Definir función de limpieza ligera
_RE_MULTI = re.compile(r'(.)\1{2,}')  # Detecta 3+ letras repetidas

def clean_text_rb(s: str) -> str:
    """
    Limpieza LIGERA para rule-based (conserva estructura original).
    
    Aplica solo:
    - Normalización NFC (forma canónica de tildes)
    - Colapso de alargamientos (holaaa → holaa)
    - Normalización de espacios
    """
    if pd.isna(s):
        return ""
    
    s = str(s).strip()
    s = unicodedata.normalize("NFC", s)  # Normaliza tildes (é = é, no e + ´)
    s = _RE_MULTI.sub(r'\1\1', s)        # holaaa → holaa (mantiene énfasis)
    s = re.sub(r"\s+", " ", s).strip()   # Colapsa espacios múltiples
    
    return s

# Aplicar limpieza ligera
dataset_base['texto_rb'] = dataset_base[text_col].map(clean_text_rb)

print(f"\n🧹 Aplicado preprocesamiento ligero (conserva tildes/mayúsculas)")
print(f"   Ejemplo antes: {dataset_base[text_col].iloc[0][:80]}...")
print(f"   Ejemplo después: {dataset_base['texto_rb'].iloc[0][:80]}...")

# Separar train y val usando índices guardados
df_train = dataset_base[dataset_base['row_id'].isin(train_indices)].copy()
df_val = dataset_base[dataset_base['row_id'].isin(val_indices)].copy()

print(f"\n📊 Splits creados:")
print(f"\nTrain ({len(df_train)} ejemplos):")
train_dist = df_train[label_col].value_counts()
for label, count in train_dist.items():
    print(f"   {label}: {count}")

print(f"\nVal ({len(df_val)} ejemplos):")
val_dist = df_val[label_col].value_counts()
for label, count in val_dist.items():
    print(f"   {label}: {count}")

✅ Splits cargados desde /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/:
   Train: 2500 ejemplos
   Val:   626 ejemplos
   Total: 3126 ejemplos en dataset_base.csv

📋 Columnas detectadas: texto='texto', label='etiqueta'

🧹 Aplicado preprocesamiento ligero (conserva tildes/mayúsculas)
   Ejemplo antes: Reposicion de medicacion 2) EXAMEN FISICO GRAL. Y GINECOLOGICO PESO ( ) TALLA ( ...
   Ejemplo después: Reposicion de medicacion 2) EXAMEN FISICO GRAL. Y GINECOLOGICO PESO ( ) TALLA ( ...

📊 Splits creados:

Train (2500 ejemplos):
   depresion: 1760
   ansiedad: 740

Val (626 ejemplos):
   depresion: 441
   ansiedad: 185

🧹 Aplicado preprocesamiento ligero (conserva tildes/mayúsculas)
   Ejemplo antes: Reposicion de medicacion 2) EXAMEN FISICO GRAL. Y GINECOLOGICO PESO ( ) TALLA ( ...
   Ejemplo después: Reposicion de medicacion 2) EXAMEN FISICO GRAL. Y GINECOLOGICO PESO ( ) TALLA ( ...

📊 Splits creados:

Train (2500 ejemplos):
   depresion: 1760
   ansiedad: 740

Val

## 2) Ejecutar fork (perfil `col`) con solo Ansiedad/Depresión

In [21]:
import sys, subprocess, yaml
from pathlib import Path

cfg_dir = FORK_PATH/'configs'
cfg_dir.mkdir(parents=True, exist_ok=True)
col_cfg = cfg_dir/'col_config.yml'
fenos_yml = cfg_dir/'fenotipos.yml'

# Forzar solo Ansiedad/Depresion en el fork
cfg = {}
if col_cfg.exists():
    cfg = yaml.safe_load(col_cfg.read_text(encoding='utf-8')) or {}
cfg['text_column'] = 'texto_rb'
col_cfg.write_text(yaml.safe_dump(cfg, allow_unicode=True), encoding='utf-8')

fen = {}
if fenos_yml.exists():
    fen = yaml.safe_load(fenos_yml.read_text(encoding='utf-8')) or {}
fen['active_concepts'] = ['Ansiedad','Depresion']
fenos_yml.write_text(yaml.safe_dump(fen, allow_unicode=True), encoding='utf-8')

cli_py = FORK_PATH/'cli.py'
main_py = FORK_PATH/'main.py'
runner = cli_py if cli_py.exists() else main_py
assert runner.exists(), "No se encontró cli.py ni main.py en el fork."

# Crear temp input solo con val set (para evaluar)
tmp_in = DATA_PATH/'ips_clean_tmp.csv'
df_val[['texto_rb', label_col]].rename(columns={'texto_rb':'texto_rb'}).to_csv(tmp_in, index=False, encoding='utf-8')

# Salidas estandarizadas (comparables)
rule_pred_csv   = DATA_PATH/'rule_based_predictions.csv'
rule_report_csv = DATA_PATH/'rule_based_classification_report.csv'
rule_eval_csv   = DATA_PATH/'rule_based_eval.csv'
rule_cm_csv     = DATA_PATH/'rule_based_confusion_matrix.csv'

cmd = [sys.executable, str(runner), '--profile','col', '--config', str(col_cfg),
       '--input', str(tmp_in), '--output', str(rule_pred_csv)]
print("CMD:", " ".join(map(str,cmd)))
ret = subprocess.run(cmd, check=False, capture_output=True, text=True)
print(ret.stdout)
if ret.returncode != 0:
    print(ret.stderr)
    raise RuntimeError(f"CLI terminó con código {ret.returncode}")

CMD: /opt/anaconda3/bin/python /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY/cli.py --profile col --config /Users/manuelnunez/Projects/psych-phenotyping-paraguay/Spanish_Psych_Phenotyping_PY/configs/col_config.yml --input /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/ips_clean_tmp.csv --output /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_predictions.csv
Components in NLP pipeline:
	- tok2vec
	- morphologizer
	- attribute_ruler
	- lemmatizer
	- medspacy_pyrush
	- medspacy_target_matcher
	- medspacy_context
Concepts included (by folder): Ansiedad, Depresion
Rule categories loaded: Abulia, Abusodesustancias, Agitacinpsicomotora, Alteracindelapercepcindepesoofiguracorporal, AngustiaMiedoTemor, Anhedonia, Animodeprimido, Animoexpansivo, Ansiedad, Apata, Apetitoaumentode, Apetitodisminucinde, Autolesin, Bajaconcentracin, Bajaenerga, Culpa, Desesperanza, Efectosadversos, Fatiga, Ideacinsuicida, Ideasdemuerte, Intent

## 3) Evaluación **binaria** y exportables

In [22]:
import pandas as pd, unicodedata as _ud
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

preds = pd.read_csv(rule_pred_csv)
if 'pred_label' not in preds.columns:
    raise ValueError("El output del fork no contiene 'pred_label'.")

def _norm_txt(s):
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = _ud.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return s

y_pred = preds['pred_label'].map(_norm_txt)
y_true = df_val[label_col].map(_norm_txt)

# En caso de que el fork devuelva algo fuera de A/D, lo mapeamos a la clase mayoritaria para evaluar binario
allowed = {'ansiedad','depresion'}
majority = y_true.value_counts().idxmax()
y_pred = y_pred.where(y_pred.isin(allowed), majority)

classes = ['depresion','ansiedad']

pd.DataFrame(classification_report(y_true, y_pred, labels=classes, output_dict=True, zero_division=0)).transpose()  .to_csv(rule_report_csv, index=True, encoding='utf-8')

pd.DataFrame([{
    'macro_f1': f1_score(y_true, y_pred, average='macro', zero_division=0),
    'macro_precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
    'macro_recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
    'n': int(len(y_true))
}]).to_csv(rule_eval_csv, index=False, encoding='utf-8')

cm = confusion_matrix(y_true, y_pred, labels=classes)
pd.DataFrame(cm, index=[f'true_{c}' for c in classes], columns=[f'pred_{c}' for c in classes]).to_csv(rule_cm_csv)

print("✅ Exportados:")
print(" - Predicciones:", rule_pred_csv)
print(" - Reporte:", rule_report_csv)
print(" - Métricas:", rule_eval_csv)
print(" - Matriz:", rule_cm_csv)

✅ Exportados:
 - Predicciones: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_predictions.csv
 - Reporte: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_classification_report.csv
 - Métricas: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_eval.csv
 - Matriz: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/rule_based_confusion_matrix.csv
